In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv


In [2]:
train=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv", index_col='date')
train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
date,,,,,
02.01.2013,0,59,22154,999.00,1.0
03.01.2013,0,25,2552,899.00,1.0
05.01.2013,0,25,2552,899.00,-1.0
06.01.2013,0,25,2554,1709.05,1.0
15.01.2013,0,25,2555,1099.00,1.0


In [3]:
train.drop(['date_block_num'], axis=1, inplace=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2935849 entries, 02.01.2013 to 03.10.2015
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   shop_id       int64  
 1   item_id       int64  
 2   item_price    float64
 3   item_cnt_day  float64
dtypes: float64(2), int64(2)
memory usage: 112.0+ MB


In [4]:
items=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
items.head(1000)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
995,"3D Action Puzzle ""Животные"" Носорог",995,67
996,"3D Action Puzzle ""Животные"" Панда",996,67
997,"3D Action Puzzle ""Животные"" Черепаха",997,67
998,"3D Action Puzzle ""Зомби "" Железнодорожник",998,67


In [5]:
train.shape

(2935849, 4)

In [6]:
test=pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv', index_col='ID')
test.head(10)

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
5,5,5039
6,5,5041
7,5,5046
8,5,5319


In [7]:
train.dropna(axis=0, subset=['item_cnt_day'], inplace=True)
y = train.item_cnt_day
train.drop(['item_cnt_day'], axis=1, inplace=True)
X = [cname for cname in train.columns if 
                train[cname].dtype in ['int64', 'float64']]

In [8]:
train_columns=train.columns.tolist()
for column in train_columns:
    print (column)
    train_columns=train.columns.tolist()
    

shop_id
item_id
item_price


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt


In [10]:

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val=train_test_split(train, y, random_state=123, test_size=0.2)

In [11]:
from sklearn.preprocessing import OneHotEncoder
object_cols = [col for col in X_train.columns if X_train[col].dtype == "float64" ]
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))


OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols ]))
OH_cols_val = pd.DataFrame(OH_encoder.transform(X_val[low_cardinality_cols ]))


OH_cols_train.index = X_train.index
OH_cols_val.index = X_val.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_val = X_val.drop(object_cols, axis=1)


X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
X_val = pd.concat([num_X_val, OH_cols_val], axis=1)

In [12]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(fit_intercept= True, normalize=True)
model2=lr.fit(X_train, y_train)
pred2=model2.predict(X_val)
print(model2.score(X_val,y_val))



0.00046568344067676737


In [13]:
from sklearn.ensemble import RandomForestRegressor
rfg= RandomForestRegressor(n_estimators=50, criterion='mse', max_depth=5, verbose=0)
model3=rfg.fit(X_train, y_train)
pred3=model3.predict(X_val)
print(model3.score(X_val, y_val))

0.17446930991433274


In [14]:
#from sklearn.model_selection import RandomizedSearchCV,KFold
#from xgboost import XGBRegressor
#params = {
#        'min_child_weight': [1, 5, 10],
#       'colsample_bytree': np.arange(0.3,0.5,0.1),
#        'colsample_bylevel': np.arange(0.3,0.5,0.1),
#        'max_depth': range(6,10),
#       'n_estimators': range(100,1000)
#        }
#xgb = XGBRegressor(learning_rate=0.04,gamma = 0,subsample = 1,silent=True, nthread=-1,n_jobs=-1)
#rand = RandomizedSearchCV(xgb, n_iter=20,refit ='MSLE', param_distributions=params, n_jobs=-1,verbose=2, random_state=50,return_train_score = True)
#model4= rand.fit(X_train, y_train)
#pred4= rand.predict(X_val)

#print(model4.score(X_val, y_val))

In [15]:
sample= pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
sample

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [16]:
final_pred=model3.predict(test)
final_pred

array([1.1670453 , 1.1670453 , 1.1670453 , ..., 1.14694104, 1.09522351,
       1.21635796])

In [17]:
output = pd.DataFrame({'ID': test.index,
                       'item_cnt_month': final_pred/30})
output.to_csv('submission.csv', index=False)

In [18]:
output

,ID,item_cnt_month
0,0,0.038902
1,1,0.038902
2,2,0.038902
3,3,0.038902
4,4,0.038902
...,...,...
214195,214195,0.034882
214196,214196,0.038231
214197,214197,0.038231
214198,214198,0.036507


In [19]:

#import random
#from sklearn.linear_model import SGDClassifier
#import numpy as np

#sg=SGDClassifier(loss='log')
#model1=sg.fit(X_train, y_train)
#model1.score(X_val, y_val)

In [20]:
#for n in range(n_iter):
 #   random.shuffle(shuffledRange)
  #  shuffleX=[X[i] for i in shuffledRange]
   # shuffleY=[y[i] for i in shuffledRange]
#    for batch in batches(range(len(shuffleX)), 10000):
 #       model1.partial_fit(shuffleX[batch[0]:batch[-1]+1],shuffleY[batch[0]:batch[-1]+1], classes=np.unique(y))